In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pywt
from scipy import signal
# S = (4500000)/24/(2**23-1) #uV/count

In [2]:
#closest stamp for 1 value
def find_closest(arr, val):
    idx = np.abs(arr - val).argmin()
    return (arr[idx] , idx)

In [3]:
#closest stamps for our dataset
def closest_stamp_values(time_stamps,events):
    closest_stamps_vals = np.zeros(len(events))
    closest_stamps_ids = np.zeros(len(events)) 
    
    event_labels = []
    
    for i in range(len(events)):
        closest_stamps_vals[i] = find_closest(time_stamps, events[i,0])[0]
        closest_stamps_ids[i] = find_closest(time_stamps, events[i,0])[1]
        event_labels.append(events[i,1])
    
    return closest_stamps_vals, closest_stamps_ids, event_labels
    

In [4]:
def remove_fixation_and_baseline(closest_stamps_ids,event_labels,eeg_data):
    session_data =  np.random.randint(100000,900000,(250,250,8))
    rc = 0 # row count 
    markers = []
    for i,image in enumerate(closest_stamps_ids):
        if event_labels[i] == 'red' or event_labels[i] == 'white':
            for j in range(5):
                if event_labels[i] == 'red':
                    markers.append(0)
                if event_labels[i] == 'white':
                    markers.append(1)
                for k in range(250):
                    if int(closest_stamps_ids[i])+j*256+250 > eeg_data.shape[0]:
                        break
                    session_data[rc][k] = eeg_data[int(closest_stamps_ids[i])+j*256:int(closest_stamps_ids[i])+j*256+250][k]
                rc = rc + 1            
    rc = 0
    return markers,session_data

In [5]:
def extract_features(session_data):
    _mean = []
    _std = []
    _skewness = []
    _kurtosis = []
    _hjorth_activity = []
    _hjorth_mobility = []
    _hjorth_complexity = []
    _a1 = []
    _a2=  []
    _a3 = []
    _b1 = []
    _b2 = []
    _b3 = []
    _c1 = []
    _c2 = []
    _c3 = []
    _d1 = []
    _d2 = []
    _d3 = []
    _e1 = []
    _e2 = []
    _e3 = []
    _f1 = []
    _f2 = []
    _f3 = []
    _g1 = []
    _g2 = []
    _g3 = []
    _h1 = []
    _h2 = []
    _h3 = []
    _bandpowers = []

    for i in range(250):
        _mean.append(np.mean(session_data[i], axis = 0))
        _std.append(np.std(session_data[i], axis = 0))
        _skewness.append(skew(session_data[i], axis = 0))
        _kurtosis.append(kurtosis(session_data[i], axis = 0))
        hjorth_activity,hjorth_mobility,hjorth_complexity = all_hjorth(session_data[i])
        _hjorth_activity.append(hjorth_activity)
        _hjorth_mobility.append(hjorth_mobility)
        _hjorth_complexity.append(hjorth_complexity)
        _wf = wavelet_features(session_data[i].T,8)
     #   _a1.append(a1)
     #   _a2.append(a2)
     #   _a3.append(a3)
     #   _b1.append(b1)
     #   _b2.append(b2)
     #   _b3.append(b3)
    #    _c1.append(c1)
    #    _c2.append(c2)
    #    _c3.append(c3)
     #   _d1.append(d1)
     #   _d2.append(d2)
     #   _d3.append(d3)
     #   _e1.append(e1)
     #   _e2.append(e2)
    #    _e3.append(e3)
     #   _f1.append(f1)
     #   _f2.append(f2)
    #    _f3.append(f3)
    #    _g1.append(g1)
    #    _g2.append(g2)
    #    _g3.append(g3)
     #   _h1.append(h1)
     #   _h2.append(h2)
    #    _h3.append(h3)
        _bandpowers.append(bandpowers(session_data[i].T))
               
               
    _mean = np.array(_mean)
    _std = np.array(_std)
    _skewness = np.array(_skewness)
    _kurtosis = np.array(_kurtosis)
    _hjorth_activity = np.array(_hjorth_activity)
    _hjorth_mobility = np.array(_hjorth_mobility)
    _hjorth_complexity = np.array(_hjorth_complexity)
    _bandpowers = np.array(_bandpowers).reshape(250,16)
    print("start")
    print(_mean.shape)
    print(_std.shape)
    print(_skewness.shape)
    print(_kurtosis.shape)
    print(_hjorth_activity.shape)
    print(_hjorth_mobility.shape)
    print(_hjorth_complexity.shape)
    print( _bandpowers.shape)
 #   print(np.array(_a1).reshape(250,1).shape)
 #   print(np.array(_a2).reshape(250,1).shape)
 #    print(np.array(_a3).reshape(250,1).shape)
 #   print(np.array(_b1).reshape(250,1).shape)
#    print(np.array(_b2).reshape(250,1).shape)
 #   print(np.array(_b3).reshape(250,1).shape)
 #   print(np.array(_c1).reshape(250,1).shape)
 #   print(np.array(_c2).reshape(250,1).shape)
 #   print(np.array(_c3).reshape(250,1).shape)
 #   print(np.array(_d1).reshape(250,1).shape)
 #   print(np.array(_d2).reshape(250,1).shape)
 #   print(np.array(_d3).reshape(250,1).shape)
 #   print(np.array(_e1).reshape(250,1).shape)
  #  print(np.array(_e2).reshape(250,1).shape)
 #   print(np.array(_e3).reshape(250,1).shape)
 #   print(np.array(_f1).reshape(250,1).shape)
 #   print(np.array(_f2).reshape(250,1).shape)
 #   print(np.array(_f3).reshape(250,1).shape)
  #  print(np.array(_g1).reshape(250,1).shape)
 #   print(np.array(_g2).reshape(250,1).shape)
 #   print(np.array(_g3).reshape(250,1).shape)
  #  print(np.array(_h1).reshape(250,1).shape)
 #   print(np.array(_h2).reshape(250,1).shape)
  #  print(np.array(_h3).reshape(250,1).shape)
    print("end")
    stat_features = np.concatenate((_mean, _std, _skewness, _kurtosis,_hjorth_activity,_hjorth_mobility,_hjorth_complexity,_bandpowers),axis=1)        
  #  stat_features = np.concatenate((_mean, _std, _skewness, _kurtosis,_hjorth_activity,_hjorth_mobility,_hjorth_complexity,np.array(_a1).reshape(250,1),np.array(_a2).reshape(250,1),np.array(_a3).reshape(250,1),np.array(_b1).reshape(250,1),np.array(_b2).reshape(250,1),np.array(_b3).reshape(250,1),np.array(_c1).reshape(250,1),np.array(_c2).reshape(250,1),np.array(_c3).reshape(250,1),np.array(_d1).reshape(250,1),np.array(_d2).reshape(250,1),np.array(_d3).reshape(250,1),np.array(_e1).reshape(250,1),np.array(_e2).reshape(250,1),np.array(_e3).reshape(250,1),np.array(_f1).reshape(250,1),np.array(_f2).reshape(250,1),np.array(_f3).reshape(250,1),np.array(_g1).reshape(250,1),np.array(_g2).reshape(250,1),np.array(_g3).reshape(250,1),np.array(_h1).reshape(250,1),np.array(_h2).reshape(250,1),np.array(_h3).reshape(250,1),_bandpowers),axis=1)
  
    return stat_features

In [6]:
def bandpowers(segment):
    features = []
    for i in range(len(segment)):
        f,Psd = signal.welch(segment[i,:], 100)
        power1 = 0
        power2 = 0
        f1 = []
        for j in range(0,len(f)):
            if(f[j]>=4 and f[j]<=13):
                power1 += Psd[j]
            if(f[j]>=14 and f[j]<=30):
                power2 += Psd[j]
        features.append(power1)
        features.append(power2)
    return np.array(features)

In [7]:
#for all the 8 channels combined
def all_hjorth(input):                     
    new_input = input.T
    all_hjorth_activity = []
    all_hjorth_mobility = []
    all_hjorth_complexity = []
    for i in range(8):
        hjorth_activity,hjorth_mobility,hjorth_complexity = hjorth(new_input[i].reshape(1,250))
        all_hjorth_activity.append(hjorth_activity)
        all_hjorth_mobility.append(hjorth_mobility)
        all_hjorth_complexity.append(hjorth_complexity)
        
    return np.array(all_hjorth_activity), np.array(all_hjorth_mobility), np.array(all_hjorth_complexity)  

In [8]:
#for 1 channel
def hjorth(input):                                             
    realinput = input
    hjorth_activity = np.zeros(len(realinput))
    hjorth_mobility = np.zeros(len(realinput))
    hjorth_diffmobility = np.zeros(len(realinput))
    hjorth_complexity = np.zeros(len(realinput))
    diff_input = np.diff(realinput)
    diff_diffinput = np.diff(diff_input)
    k = 0
    for j in realinput:
        hjorth_activity[k] = np.var(j)
        hjorth_mobility[k] = np.sqrt(np.var(diff_input[k])/hjorth_activity[k])
        hjorth_diffmobility[k] = np.sqrt(np.var(diff_diffinput[k])/np.var(diff_input[k]))
        hjorth_complexity[k] = hjorth_diffmobility[k]/hjorth_mobility[k]
        k = k+1
    return np.sum(hjorth_activity)/8, np.sum(hjorth_mobility)/8, np.sum(hjorth_complexity)/8                       #returning hjorth activity, hjorth mobility , hjorth complexity

In [15]:
####### FUNCTION TO FIND WAVELET FEATURES#######

def wavelet_features(epoch):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    wavelet_features = []
    for i in range(len(epoch)):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(len(epoch)):   
        cA_Energy.append(abs(np.sum(np.square(cA_values[x]))))
        wavelet_features.append(abs(np.sum(np.square(cA_values[x]))))
        
    for x in range(len(epoch)):      
        cD_Energy.append(abs(np.sum(np.square(cD_values[x]))))
        wavelet_features.append(abs(np.sum(np.square(cD_values[x]))))
        
    return wavelet_features

In [10]:
def autogressiveModelParametersBurg(labels):
    feature = []
    a1=[]
    a2=[]
    a3=[]
    model_order = 3
    for i in range(32):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for i in range(32):
        a1.append(feature[i][0])
        a2.append(feature[i][1])
        a3.append(feature[i][2])
        
    
    ar1=np.mean(a1).real
    ar2=np.mean(a2).real 
    ar3=np.mean(a3).real
    
    br1=np.min(a1).real
    br2=np.min(a2).real 
    br3=np.min(a3).real
    
    cr1=np.max(a1).real
    cr2=np.max(a2).real 
    cr3=np.max(a3).real
    
    
    
    return ar1,ar2,ar3,br1,br2,br3,cr1,cr2,cr3

In [12]:
def model_test_for_sessions_together(root_path,I,J,K):
    df_1 = pd.read_csv(root_path + str(I) + '\\trials.txt', header=4)
    events_df_1 = pd.read_csv(root_path + str(I) + '\\events.txt', header=None)

    df_2 = pd.read_csv(root_path + str(J) + '\\trials.txt', header=4)
    events_df_2 = pd.read_csv(root_path + str(J) + '\\events.txt', header=None)

    df_3 = pd.read_csv(root_path + str(K) + '\\trials.txt', header=4)
    events_df_3 = pd.read_csv(root_path + str(K) + '\\events.txt', header=None)
    
    df_1 = df_1[[' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2', ' EXG Channel 3', ' EXG Channel 4', ' EXG Channel 5', ' EXG Channel 6', ' EXG Channel 7', ' Timestamp']]
    df_2 = df_2[[' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2', ' EXG Channel 3', ' EXG Channel 4', ' EXG Channel 5', ' EXG Channel 6', ' EXG Channel 7', ' Timestamp']]
    df_3 = df_3[[' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2', ' EXG Channel 3', ' EXG Channel 4', ' EXG Channel 5', ' EXG Channel 6', ' EXG Channel 7', ' Timestamp']]
    
    df_1 = df_1.rename(columns={" EXG Channel 0":"Fp1", " EXG Channel 1":"Fp2", " EXG Channel 2":"C3", " EXG Channel 3":"C4", " EXG Channel 4":"P3", " EXG Channel 5":"P4", " EXG Channel 6":"O1", " EXG Channel 7":"O2"})
    df_2 = df_2.rename(columns={" EXG Channel 0":"Fp1", " EXG Channel 1":"Fp2", " EXG Channel 2":"C3", " EXG Channel 3":"C4", " EXG Channel 4":"P3", " EXG Channel 5":"P4", " EXG Channel 6":"O1", " EXG Channel 7":"O2"})
    df_3 = df_3.rename(columns={" EXG Channel 0":"Fp1", " EXG Channel 1":"Fp2", " EXG Channel 2":"C3", " EXG Channel 3":"C4", " EXG Channel 4":"P3", " EXG Channel 5":"P4", " EXG Channel 6":"O1", " EXG Channel 7":"O2"})
    
    events_1 = events_df_1.to_numpy()
    events_2 = events_df_2.to_numpy()
    events_3 = events_df_3.to_numpy()
   
    data_1 = df_1.to_numpy()
    data_2 = df_2.to_numpy()
    data_3 = df_3.to_numpy()
    
    eeg_data_1 = data_1[:,:8]
    eeg_data_2 = data_2[:,:8]
    eeg_data_3 = data_3[:,:8]
    
    time_stamp_1 = data_1[:, 8]
    time_stamp_2 = data_2[:, 8]
    time_stamp_3 = data_3[:, 8]
    
    closest_stamps_vals_1 = np.zeros(len(events_1))
    closest_stamps_ids_1 = np.zeros(len(events_1))

    closest_stamps_vals_2 = np.zeros(len(events_2))
    closest_stamps_ids_2 = np.zeros(len(events_2))

    closest_stamps_vals_3 = np.zeros(len(events_3))
    closest_stamps_ids_3 = np.zeros(len(events_3))


    event_labels_1 = []
    event_labels_2 = []
    event_labels_3 = []

    closest_stamps_vals_1, closest_stamps_ids_1, event_labels_1 = closest_stamp_values(time_stamp_1,events_1)
    closest_stamps_vals_2, closest_stamps_ids_2, event_labels_2 = closest_stamp_values(time_stamp_2,events_2)
    closest_stamps_vals_3, closest_stamps_ids_3, event_labels_3 = closest_stamp_values(time_stamp_3,events_3)
 
    
    print(len(closest_stamps_ids_1), len(event_labels_1),eeg_data_1.shape)
    markers_1, session_data_1 = remove_fixation_and_baseline(closest_stamps_ids_1,event_labels_1,eeg_data_1)
    markers_2, session_data_2 = remove_fixation_and_baseline(closest_stamps_ids_2,event_labels_2,eeg_data_2)
    markers_3, session_data_3 = remove_fixation_and_baseline(closest_stamps_ids_3,event_labels_3,eeg_data_3)
    
   
    
    markers_1 = np.array(markers_1)
    markers_2 = np.array(markers_2)
    markers_3 = np.array(markers_3)
    
    stat_features_1 = extract_features(session_data_1)
    stat_features_2 = extract_features(session_data_2)
    stat_features_3 = extract_features(session_data_3)
    
    print("stat_features_" +  str(I) + " shape=" + str(stat_features_1.shape))
    print("stat_features_" +  str(J) + " shape=" + str(stat_features_2.shape))
    print("stat_features_" +  str(K) + " shape=" + str(stat_features_3.shape))
    final_features = np.concatenate((stat_features_1, stat_features_2, stat_features_3), axis = 0)
    markers = np.concatenate((markers_1, markers_2, markers_3), axis=0)
    
    print(final_features.shape)
    print(markers.shape)
    print('...printing seassion_data_' + str(I) + ' accuracy.....................')
    diff_models_accuracy_on_features(stat_features_1,markers_1)
    print('\n\n\n...printing seassion_data_' + str(J) + ' accuracy.....................')
    diff_models_accuracy_on_features(stat_features_2,markers_2)
    print('\n\n\n...printing seassion_data_' + str(K) + ' accuracy.....................')
    diff_models_accuracy_on_features(stat_features_3,markers_3)
    
    print('\n\n\n\n\n\n\n...printing final_session_data_accuracy')
    diff_models_accuracy_on_features(final_features,markers)

In [13]:
def diff_models_accuracy_on_features(final_features,markers):
     
    X_train, X_test, y_train, y_test = train_test_split(final_features, markers, test_size=0.3, random_state=42)

    # SVM
    svm_clf = make_pipeline(StandardScaler(), SVC(kernel = 'linear'))
    svm_clf.fit(X_train, y_train)
    y_pred = svm_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('Linear SVM Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')


    # SVM
    svm2_clf = make_pipeline(StandardScaler(), SVC(kernel = 'rbf'))
    svm2_clf.fit(X_train, y_train)
    y_pred = svm2_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('RBF SVM Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')


    # RF
    rf_clf = RandomForestClassifier()
    rf_clf.fit(X_train, y_train)
    y_pred = rf_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('RF Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')
 
    # LDA

    lda_clf = LinearDiscriminantAnalysis()
    lda_clf.fit(X_train, y_train)
    y_pred = lda_clf.predict(X_test) 
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('LDA Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')


    # DT
    dt_clf = DecisionTreeClassifier()
    dt_clf = dt_clf.fit(X_train, y_train)
    y_pred = dt_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('DT Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')
 
    # NB
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('NB Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')

    # KNN
    knn_clf = KNeighborsClassifier()
    knn_clf = knn_clf.fit(X_train, y_train)
    y_pred = knn_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('KNN Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')


    # GPC
    gp_clf = GaussianProcessClassifier()
    gp_clf = gp_clf.fit(X_train, y_train)
    y_pred = gp_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('GPC Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')

   # AdaBoost
    adb_clf = AdaBoostClassifier()
    adb_clf = adb_clf.fit(X_train, y_train)
    y_pred = adb_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('AdaBoost Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')


   # QDA
    qda_clf = QuadraticDiscriminantAnalysis()
    qda_clf = qda_clf.fit(X_train, y_train)
    y_pred = qda_clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('QDA Accuracy: ' + str(accuracy_score(y_test, y_pred)*100) + '%')


In [14]:
root_path = 'C:/Users/tutan/Bi_Color/data_' 

model_test_for_sessions_together(root_path,1,2,3)
model_test_for_sessions_together(root_path,4,5,6)

102 102 (108582, 8)


c:\python37\lib\site-packages\scipy\signal\spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  .format(nperseg, input_length))


start
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 16)
end
start
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 16)
end
start
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 16)
end
stat_features_1 shape=(250, 72)
stat_features_2 shape=(250, 72)
stat_features_3 shape=(250, 72)
(750, 72)
(750,)
...printing seassion_data_1 accuracy.....................
[[14 26]
 [ 7 28]]
Linear SVM Accuracy: 56.00000000000001%
[[14 26]
 [10 25]]
RBF SVM Accuracy: 52.0%
[[17 23]
 [13 22]]
RF Accuracy: 52.0%
[[18 22]
 [10 25]]
LDA Accuracy: 57.333333333333336%
[[22 18]
 [19 16]]
DT Accuracy: 50.66666666666667%
[[ 6 34]
 [ 1 34]]
NB Accuracy: 53.333333333333336%
[[21 19]
 [13 22]]
KNN Accuracy: 57.333333333333336%
[[40  0]
 [35  0]]
GPC Accuracy: 53.333333333333336%
[[19 21]
 [16 19]]
AdaBoost Accuracy: 50.66666666666667%
[[13 27]
 [ 3 32]]
QDA Accuracy: 60.0%



...printing seassion_data_2 accuracy.....................
[[17 23]
 [17 18]]
Li

c:\python37\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


[[19 21]
 [15 20]]
RF Accuracy: 52.0%
[[20 20]
 [21 14]]
LDA Accuracy: 45.33333333333333%
[[21 19]
 [16 19]]
DT Accuracy: 53.333333333333336%
[[ 1 39]
 [ 1 34]]
NB Accuracy: 46.666666666666664%
[[18 22]
 [15 20]]
KNN Accuracy: 50.66666666666667%
[[40  0]
 [35  0]]
GPC Accuracy: 53.333333333333336%
[[15 25]
 [19 16]]
AdaBoost Accuracy: 41.333333333333336%
[[21 19]
 [10 25]]
QDA Accuracy: 61.33333333333333%



...printing seassion_data_3 accuracy.....................
[[24 16]
 [17 18]]
Linear SVM Accuracy: 56.00000000000001%
[[15 25]
 [16 19]]
RBF SVM Accuracy: 45.33333333333333%


...

[[19 21]
 [14 21]]
RF Accuracy: 53.333333333333336%
[[17 23]
 [14 21]]
LDA Accuracy: 50.66666666666667%
[[23 17]
 [16 19]]
DT Accuracy: 56.00000000000001%
[[ 5 35]
 [ 4 31]]
NB Accuracy: 48.0%
[[21 19]
 [15 20]]
KNN Accuracy: 54.666666666666664%
[[40  0]
 [35  0]]
GPC Accuracy: 53.333333333333336%
[[18 22]
 [19 16]]
AdaBoost Accuracy: 45.33333333333333%
[[ 9 31]
 [ 7 28]]
QDA Accuracy: 49.333333333333336%







...printing final_session_data_accuracy
[[41 70]
 [40 74]]
Linear SVM Accuracy: 51.11111111111111%
[[28 83]
 [34 80]]
RBF SVM Accuracy: 48.0%


c:\python37\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


[[59 52]
 [61 53]]
RF Accuracy: 49.77777777777778%
[[62 49]
 [56 58]]
LDA Accuracy: 53.333333333333336%
[[57 54]
 [50 64]]
DT Accuracy: 53.77777777777778%
[[  4 107]
 [  3 111]]
NB Accuracy: 51.11111111111111%
[[69 42]
 [53 61]]
KNN Accuracy: 57.77777777777777%
[[111   0]
 [114   0]]
GPC Accuracy: 49.333333333333336%
[[61 50]
 [48 66]]
AdaBoost Accuracy: 56.44444444444444%
[[22 89]
 [16 98]]
QDA Accuracy: 53.333333333333336%
102 102 (99196, 8)


c:\python37\lib\site-packages\scipy\signal\spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  .format(nperseg, input_length))


start
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 16)
end
start
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 16)
end
start
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 8)
(250, 16)
end
stat_features_4 shape=(250, 72)
stat_features_5 shape=(250, 72)
stat_features_6 shape=(250, 72)
(750, 72)
(750,)
...printing seassion_data_4 accuracy.....................
[[20 20]
 [15 20]]
Linear SVM Accuracy: 53.333333333333336%
[[21 19]
 [14 21]]
RBF SVM Accuracy: 56.00000000000001%
[[18 22]
 [15 20]]
RF Accuracy: 50.66666666666667%
[[19 21]
 [13 22]]
LDA Accuracy: 54.666666666666664%
[[20 20]
 [11 24]]
DT Accuracy: 58.666666666666664%
[[19 21]
 [14 21]]
NB Accuracy: 53.333333333333336%
[[22 18]
 [18 17]]
KNN Accuracy: 52.0%
[[40  0]
 [35  0]]
GPC Accuracy: 53.333333333333336%
[[22 18]
 [18 17]]
AdaBoost Accuracy: 52.0%
[[14 26]
 [ 8 27]]
QDA Accuracy: 54.666666666666664%



...printing seassion_data_5 accuracy.....................
[[19 2

In [78]:
stat_features = extract_features(session_data)

NameError: name 'session_data' is not defined